导入+打分函数

In [1]:
import re
from typing import List, Dict

def gpt_score_sadness(text: str) -> float:
    strong_sad = {"绝望", "想死", "撑不住", "抑郁", "没有希望", "活不下去"}
    medium_sad = {"孤独", "无助", "悲伤", "痛苦"}
    light_sad = {"累", "不想", "沉重", "疲惫", "烦", "难受"}

    score = 0
    score += sum(1.0 for w in strong_sad if w in text)
    score += sum(0.5 for w in medium_sad if w in text)
    score += sum(0.3 for w in light_sad if w in text)

    return min(score / 3, 1.0)  # 归一化到 0-1 分数段


滑动窗口构造函数

In [2]:
def construct_sliding_windows(dialogue: List[str], window_size: int = 5, min_chars: int = 50, max_chars: int = 150, overlap: int = 2) -> List[Dict]:
    windows = []
    i = 0
    while i < len(dialogue):
        window_text = ""
        start = i
        end = i + window_size
        last_valid = None
        for j in range(start, min(end, len(dialogue))):
            new_text = window_text + " " + dialogue[j]
            if len(new_text) > max_chars:
                break
            window_text = new_text.strip()
            if len(window_text) >= min_chars:
                last_valid = {
                    "start_round": start + 1,
                    "end_round": j + 1,
                    "text": window_text
                }
        if last_valid:
            windows.append(last_valid)
        i += (window_size - overlap)
    return windows


打分+生成html

In [3]:
def analyze_dialogue(dialogue: List[str]):
    results = []
    windows = construct_sliding_windows(dialogue)
    for win in windows:
        try:
            score = gpt_score_sadness(win['text'])
            results.append({
                "start": win["start_round"],
                "end": win["end_round"],
                "text": win["text"],
                "sadness_score": score
            })
        except Exception as e:
            print(f"Error scoring window {win['start_round']}-{win['end_round']}: {e}")
    return results


In [4]:
def render_html(results):
    html = "<html><body style='font-family:Arial;'>"
    html += "<h2>悲伤程度分析报告</h2>"
    for r in results:
        html += f"<div style='margin-bottom:20px;padding:10px;border:1px solid #ccc;border-radius:5px;'>"
        html += f"<b>轮次 {r['start']} - {r['end']}</b><br>"
        html += f"<p>{r['text']}</p>"
        html += f"<p><b>悲伤程度：</b> {r['sadness_score']:.2f}</p>"
        html += "</div>"
    html += "</body></html>"
    return html

输入数据+跑分析+展示结果

In [5]:
dialogue = [
    "我最近真的很累，什么都不想做。",
    "为什么会这样？",
    "也不知道，就是每天都不想起床。",
    "你有没有试过出去走走？",
    "走不动，感觉身体都沉重了。",
    "那你有没有和朋友聊聊？",
    "没有，我不想麻烦别人。",
    "我其实一直都觉得很孤独。",
    "也许你需要去看看心理医生。",
    "我怕别人觉得我很奇怪。"
]

results = analyze_dialogue(dialogue)
html = render_html(results)

from IPython.core.display import display, HTML
display(HTML(html))


这个是没有滑动窗口的score是怎么运行的

In [6]:
text = "我最近真的不想出门，也没什么食欲，感觉好孤独。"
print("Sadness Score:", gpt_score_sadness(text))


Sadness Score: 0.26666666666666666
